In [13]:
!pip install neo4j


[notice] A new release of pip available: 22.2.1 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
from neo4j import GraphDatabase
import logging
from neo4j.exceptions import ServiceUnavailable
import pandas as pd
import numpy as np

In [15]:
# from google.colab import drive
# drive.mount('/content/drive')

In [16]:
df = pd.read_csv('../03 Info Extraction and Triples/Updated_Data/fta_pop_gdp_cleaned_v2.csv')
df_trades=pd.read_csv('../03 Info Extraction and Triples/Updated_Data/trades_v2.csv')
# df_from = pd.read_csv('/content/drive/MyDrive/DSCI558_PJT/data/Final_Input_Files/trade_exports_from.csv')
# df_to = pd.read_csv('/content/drive/MyDrive/DSCI558_PJT/data/Final_Input_Files/trade_exports_to.csv')

In [17]:
df.columns[5:-3]

Index(['population', 'gdp_1960', 'gdp_1961', 'gdp_1962', 'gdp_1963',
       'gdp_1964', 'gdp_1965', 'gdp_1966', 'gdp_1967', 'gdp_1968', 'gdp_1969',
       'gdp_1970', 'gdp_1971', 'gdp_1972', 'gdp_1973', 'gdp_1974', 'gdp_1975',
       'gdp_1976', 'gdp_1977', 'gdp_1978', 'gdp_1979', 'gdp_1980', 'gdp_1981',
       'gdp_1982', 'gdp_1983', 'gdp_1984', 'gdp_1985', 'gdp_1986', 'gdp_1987',
       'gdp_1988', 'gdp_1989', 'gdp_1990', 'gdp_1991', 'gdp_1992', 'gdp_1993',
       'gdp_1994', 'gdp_1995', 'gdp_1996', 'gdp_1997', 'gdp_1998', 'gdp_1999',
       'gdp_2000', 'gdp_2001', 'gdp_2002', 'gdp_2003', 'gdp_2004', 'gdp_2005',
       'gdp_2006', 'gdp_2007', 'gdp_2008', 'gdp_2009', 'gdp_2010', 'gdp_2011',
       'gdp_2012', 'gdp_2013', 'gdp_2014', 'gdp_2015', 'gdp_2016', 'gdp_2017',
       'gdp_2018', 'gdp_2019'],
      dtype='object')

In [18]:
# "MERGE (p1:Trade { tradeID: $p1 }) ON CREATE SET p1.tradedValue = $p6 "

# def get_query(row):
#         str1 = 'MERGE (n1: Country { name : $n2, code : $n1, '
#         for num in df.columns[5:-3]:
#             if row[num]==row[num]:
#                 # if num != 'gdp_2021':
#                 str1+= f'{num}: $n{num}, '
#                 # else:
#                     # str1+= f'{num}: $n{num} '
#         str1 =str1[:-2]+ '})'
#         return str1

def get_query(row):
        str1 = 'MERGE (n1: Country {countryID : $n1}) ON CREATE SET n1.name = $n2,  '
        for num in df.columns[5:-3]:
            if row[num]==row[num]:
                # if num != 'gdp_2021':
                str1+= f'n1.{num}= $n{num}, '
                # else:
                    # str1+= f'{num}: $n{num} '
        str1 =str1[:-2]
        return str1

In [19]:
str1="tejas"
str1[:-1]

'teja'

In [20]:
class App:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
          # Don't forget to close the driver connection when you are finished with it
          self.driver.close()

    def trade(self, p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11, p12):
        with self.driver.session(database="neo4j") as session:
            # Write transactions allow the driver to handle retries and transient errors
            # result = 
            session.execute_write(
                self.load_trade, p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11, p12)
            # for row in result:
            #     print("Created hasCountryCode between: {p1}, {p2}".format(p1=row['p1'], p2=row['p2']))

    @staticmethod
    def load_trade(tx, p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11, p12):
        # To learn more about the Cypher syntax, see https://neo4j.com/docs/cypher-manual/current/
        # The Reference Card is also a good resource for keywords https://neo4j.com/docs/cypher-refcard/current/
        # query = (
        #     "MATCH (p2:Country { countryID: $p2 }), "
        #     " (p3:Country { countryID: $p3 }) "
        #     "MERGE (p1:Trade { tradeID: $p1 }) ON CREATE SET p1.tradedValue = $p6 "
        #     "MERGE (p1)-[:exportedFrom]->(p2) "
        #     "MERGE (p1)-[:exportedTo]->(p3) "
        #     "MERGE (p4:Year { year: $p4 })"
        #     "MERGE (p5:Currency { Currency: $p5 })"
        #     "MERGE (p8:Product { productID: $p8,product:$p7 })"
        #     "MERGE (p10:Category { categoryID: $p10}) ON CREATE SET p10.Category = $p9 "
        #     "MERGE (p12:Section { sectionID: $p12}) ON CREATE SET p12.section = $p11 "
        #     "MERGE (p1)-[:tradedYear]->(p4) "
        #     "MERGE (p1)-[:tradedCurrency]->(p5) "
        #     "MERGE (p1)-[:tradedProduct]->(p8) "
        #     "MERGE (p8)-[:hasCategory]->(p10) "
        #     "MERGE (p10)-[:hasSection]->(p12) "
        # )

        query1=(
            "MERGE (p1:Trade { tradeID: $p1 }) ON CREATE SET p1.tradedValue = $p6 "
            "MERGE (p4:Year { year: $p4 })"
            "MERGE (p5:Currency { Currency: $p5 })"
            "MERGE (p8:Product { productID: $p8 }) ON CREATE SET p8.product=$p7 "
            "MERGE (p10:Category { categoryID: $p10}) ON CREATE SET p10.Category = $p9 "
        )
        query2=(
            "MERGE (p12:Section { sectionID: $p12}) ON CREATE SET p12.section = $p11 "
        )


        # result = tx.run(query, country=p1_name, country_code=p2_code)
        tx.run(query1, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5, p6=p6, p7=p7, p8=p8, p9=p9, p10=p10, p11=p11, p12=p12)
        # tx.run(query, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5, p6=p6, p7=p7, p8=p8, p9=p9, p10=p10, p11=p11, p12=p12)
        tx.run(query2, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5, p6=p6, p7=p7, p8=p8, p9=p9, p10=p10, p11=p11, p12=p12)

    def trade_rel(self, p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11, p12):
        with self.driver.session(database="neo4j") as session:
            # Write transactions allow the driver to handle retries and transient errors
            # result = 
            session.execute_write(
                self.load_trade_rel, p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11, p12)
            # for row in result:
            #     print("Created hasCountryCode between: {p1}, {p2}".format(p1=row['p1'], p2=row['p2']))

    @staticmethod
    def load_trade_rel(tx, p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11, p12):
        # To learn more about the Cypher syntax, see https://neo4j.com/docs/cypher-manual/current/
        # The Reference Card is also a good resource for keywords https://neo4j.com/docs/cypher-refcard/current/
        # query = (
        #     "MATCH (p2:Country { countryID: $p2 }), "
        #     " (p3:Country { countryID: $p3 }) "
        #     "MERGE (p1:Trade { tradeID: $p1 }) ON CREATE SET p1.tradedValue = $p6 "
        #     "MERGE (p1)-[:exportedFrom]->(p2) "
        #     "MERGE (p1)-[:exportedTo]->(p3) "
        #     "MERGE (p4:Year { year: $p4 })"
        #     "MERGE (p5:Currency { Currency: $p5 })"
        #     "MERGE (p8:Product { productID: $p8}) ON CREATE SET p8.product=$p7 "
        #     "MERGE (p10:Category { categoryID: $p10}) ON CREATE SET p10.Category = $p9 "
        #     "MERGE (p12:Section { sectionID: $p12}) ON CREATE SET p12.section = $p11 "
        #     "MERGE (p1)-[:tradedYear]->(p4) "
        #     "MERGE (p1)-[:tradedCurrency]->(p5) "
        #     "MERGE (p1)-[:tradedProduct]->(p8) "
        #     "MERGE (p8)-[:hasCategory]->(p10) "
        #     "MERGE (p10)-[:hasSection]->(p12) "
        # )

        query3=(
            "MATCH (p1:Trade { tradeID: $p1 }), (p2:Country { countryID: $p2 }) "
            "MERGE (p1)-[:exportedFrom]->(p2) "
            )

        query4=(
            "MATCH (p1:Trade { tradeID: $p1 }), (p3:Country { countryID: $p3 }) "
            "MERGE (p1)-[:exportedTo]->(p3) "
            )

        query5=(
            "MATCH (p1:Trade { tradeID: $p1 }), (p4:Year { year: $p4 }) "
            "MERGE (p1)-[:tradedYear]->(p4) "
            )

        query6=(
            "MATCH (p1:Trade { tradeID: $p1 }), (p5:Currency { Currency: $p5 }) "
            "MERGE (p1)-[:tradedCurrency]->(p5) "
            )

        query7=(
            "MATCH (p1:Trade { tradeID: $p1 }), (p8:Product { productID: $p8 }) "
            "MERGE (p1)-[:tradedProduct]->(p8) "
            )

        query8=(
            "MATCH (p8:Product { productID: $p8 }), (p10:Category { categoryID: $p10 }) "
            "MERGE (p8)-[:hasCategory]->(p10) "
            )

        query9=(
            "MATCH (p12:Section { sectionID: $p12}), (p10:Category { categoryID: $p10 }) "
            "MERGE (p10)-[:hasSection]->(p12) "
            )
        # print(query3)
        # result = tx.run(query, country=p1_name, country_code=p2_code)
        tx.run(query3, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5, p6=p6, p7=p7, p8=p8, p9=p9, p10=p10, p11=p11, p12=p12)
        tx.run(query4, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5, p6=p6, p7=p7, p8=p8, p9=p9, p10=p10, p11=p11, p12=p12)
        tx.run(query5, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5, p6=p6, p7=p7, p8=p8, p9=p9, p10=p10, p11=p11, p12=p12)
        tx.run(query6, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5, p6=p6, p7=p7, p8=p8, p9=p9, p10=p10, p11=p11, p12=p12)
        tx.run(query7, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5, p6=p6, p7=p7, p8=p8, p9=p9, p10=p10, p11=p11, p12=p12)
        tx.run(query8, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5, p6=p6, p7=p7, p8=p8, p9=p9, p10=p10, p11=p11, p12=p12)
        tx.run(query9, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5, p6=p6, p7=p7, p8=p8, p9=p9, p10=p10, p11=p11, p12=p12)
        

    def FTA(self, p1, p2, p3, p4, p5, p6):
        with self.driver.session(database="neo4j") as session:
            # Write transactions allow the driver to handle retries and transient errors
            # result = 
            session.execute_write(
                self.load_fta, p1, p2, p3, p4, p5, p6)
            # for row in result:
            #     print("Created hasCountryCode between: {p1}, {p2}".format(p1=row['p1'], p2=row['p2']))

    @staticmethod
    def load_fta(tx, p1, p2, p3, p4, p5, p6):
        # To learn more about the Cypher syntax, see https://neo4j.com/docs/cypher-manual/current/
        # The Reference Card is also a good resource for keywords https://neo4j.com/docs/cypher-refcard/current/
        # print(p1, p2, p3, p4, p5, p6)
        # query = (

        #     "MATCH (p5:Country { countryID: $p5 }), "
        #     " (p6:Country { countryID: $p6 }) "
        #     "MERGE (p1:FTA { ftaID: $p1 }) ON CREATE SET p1.has_fta = $p2, p1.signed_date=$p3, p1.date_inforce=$p4 "
        #     "CREATE (p1)-[:hasFTA]->(p5) "
        #     "CREATE (p1)-[:hasFTA]->(p6) "
            
        # )
        print(p4)
        query1 = (
            "MERGE (p1:FTA { ftaID: $p1 }) ON CREATE SET p1.has_fta = $p2, p1.signed_date=$p3, p1.date_inforce=$p4 "
        )

        query2=(
            "MATCH (p1:FTA { ftaID: $p1 }), (p5:Country { countryID: $p5 }) "
            "MERGE (p1)-[:hasFTA]->(p5) "
            )
        # query2 = (
        #     "MATCH (p5:Country { countryID: $p5 }), (p1:FTA { ftaID: $p1 }) MERGE (p1)-[:hasFTA]->(p5) "
        # )

        query3 = (
            "MATCH (p6:Country { countryID: $p6 }), (p1:FTA { ftaID: $p1 }) MERGE (p1)-[:hasFTA]->(p6) "
        )
        # result = tx.run(query, country=p1_name, country_code=p2_code)
        tx.run(query1, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5, p6=p6)
        tx.run(query2, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5, p6=p6)
        tx.run(query3, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5, p6=p6)
        # tx.run(query, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5, p6=p6)

    def create_country_node(self, row, has_fta):
        with self.driver.session(database="neo4j") as session:
            session.execute_write(self._create_country_node, row, has_fta)

    

    @staticmethod
    def _create_country_node(tx, row, has_fta):
        # query = (
        #     "CREATE (n1:Country { name: $n2, population: $n3, code: $n1, gdp_1960: $n1960, gdp_1961: $n1961, gdp_1962: $n1962, gdp_1963: $n1963, gdp_1964: $n1964, gdp_1965: $n1965, gdp_1966: $n1966, gdp_1967: $n1967, gdp_1968: $n1968, gdp_1969: $n1969, gdp_1970: $n1970, gdp_1971: $n1971, gdp_1972: $n1972, gdp_1973: $n1973, gdp_1974: $n1974, gdp_1975: $n1975, gdp_1976: $n1976, gdp_1977: $n1977, gdp_1978: $n1978, gdp_1979: $n1979, gdp_1980: $n1980, gdp_1981: $n1981, gdp_1982: $n1982, gdp_1983: $n1983, gdp_1984: $n1984, gdp_1985: $n1985, gdp_1986: $n1986, gdp_1987: $n1987, gdp_1988: $n1988, gdp_1989: $n1989, gdp_1990: $n1990, gdp_1991: $n1991, gdp_1992: $n1992, gdp_1993: $n1993, gdp_1994: $n1994, gdp_1995: $n1995, gdp_1996: $n1996, gdp_1997: $n1997, gdp_1998: $n1998, gdp_1999: $n1999, gdp_2000: $n2000, gdp_2001: $n2001, gdp_2002: $n2002, gdp_2003: $n2003, gdp_2004: $n2004, gdp_2005: $n2005, gdp_2006: $n2006, gdp_2007: $n2007, gdp_2008: $n2008, gdp_2009: $n2009, gdp_2010: $n2010, gdp_2011: $n2011, gdp_2012: $n2012, gdp_2013: $n2013, gdp_2014: $n2014, gdp_2015: $n2015, gdp_2016: $n2016, gdp_2017: $n2017, gdp_2018: $n2018, gdp_2019: $n2019, gdp_2020: $n2020, gdp_2021: $n2021 }) "
        # )
        # query=()
        query=(get_query(row))
        # print(query)
        # query = (
        #     "CREATE (n1:CountryCode { code: $n1, gdp_1960: $n1960, gdp_1961: $n1961, gdp_1962: $n1962, gdp_1963: $n1963, gdp_1964: $n1964, gdp_1965: $n1965, gdp_1966: $n1966, gdp_1967: $n1967, gdp_1968: $n1968, gdp_1969: $n1969, gdp_1970: $n1970, gdp_1971: $n1971, gdp_1972: $n1972, gdp_1973: $n1973, gdp_1974: $n1974, gdp_1975: $n1975, gdp_1976: $n1976, gdp_1977: $n1977, gdp_1978: $n1978, gdp_1979: $n1979, gdp_1980: $n1980, gdp_1981: $n1981, gdp_1982: $n1982, gdp_1983: $n1983, gdp_1984: $n1984, gdp_1985: $n1985, gdp_1986: $n1986, gdp_1987: $n1987, gdp_1988: $n1988, gdp_1989: $n1989, gdp_1990: $n1990, gdp_1991: $n1991, gdp_1992: $n1992, gdp_1993: $n1993, gdp_1994: $n1994, gdp_1995: $n1995, gdp_1996: $n1996, gdp_1997: $n1997, gdp_1998: $n1998, gdp_1999: $n1999, gdp_2000: $n2000, gdp_2001: $n2001, gdp_2002: $n2002, gdp_2003: $n2003, gdp_2004: $n2004, gdp_2005: $n2005, gdp_2006: $n2006, gdp_2007: $n2007, gdp_2008: $n2008, gdp_2009: $n2009, gdp_2010: $n2010, gdp_2011: $n2011, gdp_2012: $n2012, gdp_2013: $n2013, gdp_2014: $n2014, gdp_2015: $n2015, gdp_2016: $n2016, gdp_2017: $n2017, gdp_2018: $n2018, gdp_2019: $n2019, gdp_2020: $n2020, gdp_2021: $n2021 }) "
        #     "CREATE (n2:Country { name: $n2 }) "
        #     "CREATE (n3:Population { population: $n3 }) "
        #     "CREATE (n1)-[:hasCountryName]->(n2) "
        #     "CREATE (n1)-[:hasPopulation]->(n3) "
        # )
        if has_fta:
            query += (
            "CREATE (n4:SignedDate { date: $n4 }) "
            "CREATE (n5:DateInForce { date: $n5 }) "
            "CREATE (n1)-[:hasSignedDate]->(n4) "
            "CREATE (n1)-[:isInForceOn]->(n5) "
            )
# ngdp_1993
        tx.run((query), n1=row['country_code'], n2=row['country'], npopulation=row['population'],
                n4=row['signed_year'], n5=row['inforced_year'],
                ngdp_1960=row['gdp_1960'],  ngdp_1961=row['gdp_1961'],  ngdp_1962=row['gdp_1962'],  ngdp_1963=row['gdp_1963'],  ngdp_1964=row['gdp_1964'],  ngdp_1965=row['gdp_1965'],
                ngdp_1966=row['gdp_1966'],  ngdp_1967=row['gdp_1967'],  ngdp_1968=row['gdp_1968'],  ngdp_1969=row['gdp_1969'],  ngdp_1970=row['gdp_1970'],  ngdp_1971=row['gdp_1971'],
                ngdp_1972=row['gdp_1972'],  ngdp_1973=row['gdp_1973'],  ngdp_1974=row['gdp_1974'],  ngdp_1975=row['gdp_1975'],  ngdp_1976=row['gdp_1976'],  ngdp_1977=row['gdp_1977'], 
                ngdp_1978=row['gdp_1978'],  ngdp_1979=row['gdp_1979'],  ngdp_1980=row['gdp_1980'],  ngdp_1981=row['gdp_1981'],  ngdp_1982=row['gdp_1982'],  ngdp_1983=row['gdp_1983'], 
                ngdp_1984=row['gdp_1984'],  ngdp_1985=row['gdp_1985'],  ngdp_1986=row['gdp_1986'],  ngdp_1987=row['gdp_1987'],  ngdp_1988=row['gdp_1988'],  ngdp_1989=row['gdp_1989'],
                ngdp_1990=row['gdp_1990'],  ngdp_1991=row['gdp_1991'],  ngdp_1992=row['gdp_1992'],  ngdp_1993=row['gdp_1993'],  ngdp_1994=row['gdp_1994'],  ngdp_1995=row['gdp_1995'], 
                ngdp_1996=row['gdp_1996'],  ngdp_1997=row['gdp_1997'],  ngdp_1998=row['gdp_1998'],  ngdp_1999=row['gdp_1999'],  ngdp_2000=row['gdp_2000'],  ngdp_2001=row['gdp_2001'],
                ngdp_2002=row['gdp_2002'],  ngdp_2003=row['gdp_2003'],  ngdp_2004=row['gdp_2004'],  ngdp_2005=row['gdp_2005'],  ngdp_2006=row['gdp_2006'],  ngdp_2007=row['gdp_2007'],
                ngdp_2008=row['gdp_2008'],  ngdp_2009=row['gdp_2009'],  ngdp_2010=row['gdp_2010'],  ngdp_2011=row['gdp_2011'],  ngdp_2012=row['gdp_2012'],  ngdp_2013=row['gdp_2013'],
                ngdp_2014=row['gdp_2014'],  ngdp_2015=row['gdp_2015'],  ngdp_2016=row['gdp_2016'],  ngdp_2017=row['gdp_2017'],  ngdp_2018=row['gdp_2018'],  ngdp_2019=row['gdp_2019'],
                ngdp_2020=row['gdp_2020'],  ngdp_2021=row['gdp_2021']  )


In [21]:
df['signed_date'].fillna(value=pd.to_datetime('12/31/2030'), inplace=True)
df['signed_year'] = pd.DatetimeIndex(df['signed_date']).year
df['date_inforce'].fillna(value=pd.to_datetime('12/31/2030'), inplace=True)
df['inforced_year'] = pd.DatetimeIndex(df['date_inforce']).year
df['country_code']=df['country_code'].str.lower()
df['FTA_id']="fta_"+df['FTA_id'].astype(str)
df.head(2)

,country,country_code,has_fta,signed_date,date_inforce,population,gdp_1960,gdp_1961,gdp_1962,gdp_1963,...,gdp_2015,gdp_2016,gdp_2017,gdp_2018,gdp_2019,gdp_2020,gdp_2021,FTA_id,signed_year,inforced_year
0,Aruba,abw,False,2030-12-31 00:00:00,2030-12-31 00:00:00,"105,366",NaN,NaN,NaN,NaN,...,2.963128e+09,2.983799e+09,3.092179e+09,3.202235e+09,3.310056e+09,2.496648e+09,NaN,fta_0,2030,2030
1,Afghanistan,afg,False,2030-12-31 00:00:00,2030-12-31 00:00:00,"36,296,113",537777811.1,548888895.6,546666677.8,751111191.1,...,1.913421e+10,1.811656e+10,1.875347e+10,1.805323e+10,1.879945e+10,2.011614e+10,NaN,fta_1,2030,2030


In [22]:
df_trades['trade_id'] = df_trades['trade_id'].str.replace('ported','')
df_trades['trade_id'] = df_trades['trade_id'].str.lower()
df_trades.head(5)

,exports_from,exports_to,exports_year,section_id,section_name,category_id,category_name,item_id,item_name,trade_value,trade_id,currency
0,usa,afg,2000,2,Vegetable Products,208,"Fruit and nuts, edible; peel of citrus fruit o...",20806,Grapes,57330.0,ex_0,US$
1,usa,afg,2000,2,Vegetable Products,210,Cereals,21001,Wheat,5148038.0,ex_1,US$
2,usa,afg,2000,4,Foodstuffs,419,"Preparations of cereals, flour, starch or milk...",41905,Baked Goods,5493.0,ex_2,US$
3,usa,afg,2000,4,Foodstuffs,420,"Preparations of vegetables, fruit, nuts or oth...",42005,Other Processed Vegetables,15266.0,ex_3,US$
4,usa,afg,2000,5,Mineral Products,526,"Ores, slag and ash",52615,"Niobium, Tantalum, Vanadium and Zirconium Ore",61332.0,ex_4,US$


In [23]:
df.iloc[70]

country                       Guinea
country_code                     gin
has_fta                        False
signed_date      2030-12-31 00:00:00
date_inforce     2030-12-31 00:00:00
                        ...         
gdp_2020               14169626010.0
gdp_2021               15850520384.0
FTA_id                        fta_71
signed_year                     2030
inforced_year                   2030
Name: 70, Length: 71, dtype: object

In [24]:
if __name__ == "__main__":
    # Aura queries use an encrypted connection using the "neo4j+s" URI scheme
    uri = "neo4j+s://50728551.databases.neo4j.io:7687"
    user = "neo4j"
    password = "4qdJFRzdAU9l2qWCCizkH163hvQ9UD9j4rk3sYndmQs"
    app = App(uri, user, password)
    df_trades=df_trades.head(25)

    
    for idx, row in df.iterrows():
      if row['country_code']=="usa" or row['country_code']=="afg":  
        app.create_country_node(row, False)
    
    for idx, row in df.iterrows():
      if row['country_code']=="usa" or row['country_code']=="afg":  
        app.FTA(row['FTA_id'], row['has_fta'], row['signed_year'],row['inforced_year'],row['country_code'],"usa")


    for idx, row in df_trades.iterrows():
      app.trade(row['trade_id'], row['exports_from'], row['exports_to'],row['exports_year'],row['currency'],row['trade_value'],row['item_name'],row['item_id'],row['category_name'],row['category_id'],row['section_name'],row['section_id'])
    
    
    for idx, row in df_trades.iterrows():
      app.trade_rel(row['trade_id'], row['exports_from'], row['exports_to'],row['exports_year'],row['currency'],row['trade_value'],row['item_name'],row['item_id'],row['category_name'],row['category_id'],row['section_name'],row['section_id'])
    
    app.close()

2030
2030
